In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation,Dropout
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization

In [1]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 24.5 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=12bcd2fdfc9ff1c270d70bde026b55dfb40a349573b60c34cda7eb4f9abb0af9
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [3]:
# Get Data
import tflearn.datasets.oxflower17 as oxflower17
x, y = oxflower17.load_data(one_hot=True)

Instructions for updating:
non-resource variables are not supported in the long term


100.0% 60276736 / 60270631


Succesfully downloaded 17flowers.tgz 60270631 bytes.
File Extracted
Starting to parse images...
Parsing Done!


In [4]:
x

array([[[[0.07058824, 0.02745098, 0.04313726],
         [0.04313726, 0.        , 0.01568628],
         [0.05098039, 0.00784314, 0.02352941],
         ...,
         [0.22745098, 0.1254902 , 0.12941177],
         [0.19215687, 0.10588235, 0.10196079],
         [0.14117648, 0.0627451 , 0.05882353]],

        [[0.07058824, 0.02745098, 0.04313726],
         [0.04313726, 0.        , 0.01568628],
         [0.05098039, 0.00784314, 0.02352941],
         ...,
         [0.2       , 0.10980392, 0.10980392],
         [0.16470589, 0.08627451, 0.08235294],
         [0.11764706, 0.03921569, 0.03529412]],

        [[0.0627451 , 0.01960784, 0.03529412],
         [0.03921569, 0.        , 0.01176471],
         [0.05098039, 0.00784314, 0.02352941],
         ...,
         [0.16862746, 0.09411765, 0.09019608],
         [0.14117648, 0.07450981, 0.06666667],
         [0.09411765, 0.03137255, 0.02352941]],

        ...,

        [[0.5882353 , 0.62352943, 0.41960785],
         [0.6313726 , 0.6666667 , 0.4745098 ]

In [5]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
x.shape

(1360, 224, 224, 3)

In [7]:
y.shape

(1360, 17)

In [8]:
# Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))

# Pooling 
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())


# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())

# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 activation (Activation)     (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 activation_1 (Activation)   (None, 26, 26, 256)       0

In [9]:
# Compile 
opt = tf.optimizers.Adam(learning_rate = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [13]:
# Train
model.fit(x, y, batch_size=64, epochs=100, verbose=1,validation_split=0.2, shuffle=True)

Train on 1088 samples, validate on 272 samples
Epoch 1/100
1088/1088 [==============================] - 1s 1ms/sample - loss: 0.4556 - acc: 0.8355 - val_loss: 1.4867 - val_acc: 0.5809
Epoch 2/100
1088/1088 [==============================] - 1s 1ms/sample - loss: 0.4205 - acc: 0.8428 - val_loss: 1.7747 - val_acc: 0.5699
Epoch 3/100
1088/1088 [==============================] - 1s 1ms/sample - loss: 0.3996 - acc: 0.8695 - val_loss: 1.8150 - val_acc: 0.5368
Epoch 4/100
1088/1088 [==============================] - 1s 1ms/sample - loss: 0.4406 - acc: 0.8493 - val_loss: 2.6064 - val_acc: 0.4118
Epoch 5/100
1088/1088 [==============================] - 1s 1ms/sample - loss: 0.4861 - acc: 0.8382 - val_loss: 2.0791 - val_acc: 0.5037
Epoch 6/100
1088/1088 [==============================] - 1s 1ms/sample - loss: 0.3769 - acc: 0.8732 - val_loss: 1.6984 - val_acc: 0.5735
Epoch 7/100
1088/1088 [==============================] - 1s 1ms/sample - loss: 0.4960 - acc: 0.8336 - val_loss: 1.6173 - val_acc: 0